# Cybersecurity Threat Detection

### Abstract

In this project, we address the critical task of anomaly detection in network traffic using the UNSW-NB15 dataset. Network anomalies can indicate malicious activities such as cyber-attacks, making their detection vital for cybersecurity.

We preprocess the data by encoding categorical variables and scaling features to normalize the dataset. To enhance model performance and reduce computational complexity, we employ feature selection using a Random Forest classifier to identify the most significant features. We implement and compare four machine learning models: Random Forest, XGBoost, LightGBM, and a Neural Network. Each model undergoes hyperparameter tuning to optimize its performance. The models are evaluated using classification metrics and ROC curves to determine their effectiveness in accurately detecting anomalies within the network traffic data.

### Load Data and Import Libraries

In [ ]:
# Import Necessary Libraries
import pandas as pd
import numpy as np

# For Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# For Models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# For Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold

# For Evaluation
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Load the training and testing datasets
train_data = pd.read_csv('Data/UNSW_NB15_training-set.csv')
test_data = pd.read_csv('Data/UNSW_NB15_testing-set.csv')

# Combine train and test data for consistent preprocessing
full_data = pd.concat([train_data, test_data], ignore_index=True)

# Remove 'id' and 'attack_cat' columns if present
full_data.drop(columns=['id', 'attack_cat'], errors='ignore', inplace=True)


### Data Cleaning

In [ ]:
# Identify categorical columns
categorical_cols = ['proto', 'service', 'state']

# Encode categorical variables using LabelEncoder
for col in categorical_cols:
    le = LabelEncoder()
    full_data[col] = le.fit_transform(full_data[col])

# Separate the target variable 'label' from features
labels = full_data['label']
features = full_data.drop('label', axis=1)

# Scale features using StandardScaler for normalization
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


### Preparing the Data

In [ ]:
# Determine the original training set size
train_size = train_data.shape[0]

# Split features and labels back into training and test sets
X = features_scaled[:train_size]
X_test = features_scaled[train_size:]
y = labels[:train_size].values
y_test = labels[train_size:].values

# Further split the training data into training and validation sets
X_train_full, X_val, y_train_full, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


### Feature Selection

In [ ]:
# Use a Random Forest to compute feature importances
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_selector.fit(X_train_full, y_train_full)
importances = rf_selector.feature_importances_

# Get indices of features sorted by importance
indices = np.argsort(importances)[::-1]

# Select the top N features based on importance
top_n = 40
selected_indices = indices[:top_n]

# Subset the training, validation, and test data with selected features
X_train = X_train_full[:, selected_indices]
X_val = X_val[:, selected_indices]
X_test_selected = X_test[:, selected_indices]


### Model Implementation

#### A. Random Forest Classifier

In [ ]:
print("Training Random Forest Classifier...")

# Define the hyperparameter grid for RandomizedSearchCV
param_dist_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 20, 30, 40],
    'max_features': ['sqrt'],  # Use 'sqrt' to avoid deprecation warnings
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Initialize the Random Forest Classifier with balanced class weights
rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

# Perform hyperparameter tuning using RandomizedSearchCV
random_search_rf = RandomizedSearchCV(
    estimator=rf_classifier,
    param_distributions=param_dist_rf,
    n_iter=10,
    scoring='f1_macro',
    cv=StratifiedKFold(n_splits=3),
    random_state=42,
    n_jobs=-1
)

# Fit the model to the training data
random_search_rf.fit(X_train, y_train_full)
best_rf = random_search_rf.best_estimator_
print("Best parameters found for Random Forest:", random_search_rf.best_params_)

# Retrain the best model on the training data
best_rf.fit(X_train, y_train_full)

# Predict on the validation set
y_val_pred_rf = best_rf.predict(X_val)
print("\nRandom Forest Validation Performance:")
print(classification_report(y_val, y_val_pred_rf))

# Predict on the test set
y_pred_rf = best_rf.predict(X_test_selected)


#### B. XGBoost Classifier

In [ ]:
print("\nTraining XGBoost Classifier...")

# Define the hyperparameter grid for XGBoost
param_dist_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [6, 10],
    'learning_rate': [0.1, 0.01],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'scale_pos_weight': [1, 2, 5]
}

# Initialize the XGBoost Classifier
xgb_classifier = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_jobs=-1,
    random_state=42
)

# Perform hyperparameter tuning using RandomizedSearchCV
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_classifier,
    param_distributions=param_dist_xgb,
    n_iter=10,
    scoring='f1_macro',
    cv=StratifiedKFold(n_splits=3),
    random_state=42,
    n_jobs=-1
)

# Fit the model to the training data
random_search_xgb.fit(X_train, y_train_full)
best_xgb = random_search_xgb.best_estimator_
print("Best parameters found for XGBoost:", random_search_xgb.best_params_)

# Predict on the validation set
y_val_pred_xgb = best_xgb.predict(X_val)
print("\nXGBoost Validation Performance:")
print(classification_report(y_val, y_val_pred_xgb))

# Predict on the test set
y_pred_xgb = best_xgb.predict(X_test_selected)



#### C. LightGBM Classifier

In [ ]:
print("\nTraining LightGBM Classifier...")

# Define the hyperparameter grid for LightGBM
param_dist_lgb = {
    'n_estimators': [100, 200],
    'num_leaves': [31, 50],
    'learning_rate': [0.1, 0.01],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'class_weight': ['balanced', None]
}

# Initialize the LightGBM Classifier
lgb_classifier = LGBMClassifier(
    objective='binary',
    random_state=42,
    n_jobs=-1
)

# Perform hyperparameter tuning using RandomizedSearchCV
random_search_lgb = RandomizedSearchCV(
    estimator=lgb_classifier,
    param_distributions=param_dist_lgb,
    n_iter=10,
    scoring='f1_macro',
    cv=StratifiedKFold(n_splits=3),
    random_state=42,
    n_jobs=-1
)

# Fit the model to the training data
random_search_lgb.fit(X_train, y_train_full)
best_lgb = random_search_lgb.best_estimator_
print("Best parameters found for LightGBM:", random_search_lgb.best_params_)

# Predict on the validation set
y_val_pred_lgb = best_lgb.predict(X_val)
print("\nLightGBM Validation Performance:")
print(classification_report(y_val, y_val_pred_lgb))

# Predict on the test set
y_pred_lgb = best_lgb.predict(X_test_selected)



#### D. Neural Network Classifier

In [ ]:
print("\nTraining Neural Network Classifier...")

# Initialize the neural network model
model_nn = Sequential()
model_nn.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_nn.add(Dropout(0.5))
model_nn.add(Dense(32, activation='relu'))
model_nn.add(Dropout(0.5))
model_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the neural network
model_nn.fit(X_train, y_train_full, epochs=10, batch_size=256, validation_data=(X_val, y_val), verbose=0)

# Predict probabilities on the validation set
y_val_pred_nn_prob = model_nn.predict(X_val)
# Convert probabilities to binary predictions
y_val_pred_nn = (y_val_pred_nn_prob > 0.5).astype(int).reshape(-1)
print("\nNeural Network Validation Performance:")
print(classification_report(y_val, y_val_pred_nn))

# Predict on the test set
y_pred_nn_prob = model_nn.predict(X_test_selected)
y_pred_nn = (y_pred_nn_prob > 0.5).astype(int).reshape(-1)


### Model Evaluation and Comparison

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name} Evaluation:")
    print(classification_report(y_true, y_pred))

    # Plot confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
    plt.title(f"{model_name} Confusion Matrix")
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # Compute ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    return fpr, tpr, roc_auc


In [ ]:
# Initialize plot for ROC curves
plt.figure(figsize=(8, 6))

# Evaluate Random Forest
fpr_rf, tpr_rf, roc_auc_rf = evaluate_model(y_test, y_pred_rf, "Random Forest")
plt.plot(fpr_rf, tpr_rf, label='Random Forest ROC (area = %0.2f)' % roc_auc_rf)

# Evaluate XGBoost
fpr_xgb, tpr_xgb, roc_auc_xgb = evaluate_model(y_test, y_pred_xgb, "XGBoost")
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost ROC (area = %0.2f)' % roc_auc_xgb)

# Evaluate LightGBM
fpr_lgb, tpr_lgb, roc_auc_lgb = evaluate_model(y_test, y_pred_lgb, "LightGBM")
plt.plot(fpr_lgb, tpr_lgb, label='LightGBM ROC (area = %0.2f)' % roc_auc_lgb)

# Evaluate Neural Network
fpr_nn, tpr_nn, roc_auc_nn = evaluate_model(y_test, y_pred_nn, "Neural Network")
plt.plot(fpr_nn, tpr_nn, label='Neural Network ROC (area = %0.2f)' % roc_auc_nn)

# Finalize ROC curve plot
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend()
plt.show()


### Results

*High Recall for Anomalies*
\
All models demonstrated a high recall for detecting anomalies (Class 1), with values ranging from 0.98 to 0.99. This indicates that the models are proficient at identifying anomalous network traffic, which is crucial for cybersecurity applications where missing an anomaly could have severe consequences.
\
\
*Lower Recall for Normal Traffic*
\
The recall for normal traffic (Class 0) was significantly lower across all models, particularly for the Neural Network and XGBoost classifiers. This suggests that the models are misclassifying a substantial portion of normal traffic as anomalies, leading to a higher false-positive rate.
\
\
*Random Forest and LightGBM Performance*
\
The Random Forest and LightGBM classifiers achieved a better balance between precision and recall for normal traffic compared to the other models. With recalls of 0.76 and 0.74, respectively, they misclassified fewer normal instances as anomalies while maintaining high anomaly detection rates.
\
\
*Overall Accuracy and F1-Scores*
\
The overall accuracy of the models ranged from 0.83 to 0.88, with the Random Forest classifier achieving the highest accuracy. The F1-scores for anomalies were consistently high, reflecting the models' effectiveness in correctly identifying anomalies.